In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel # Importing AutoModel
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
import numpy as np
from transformers import LongformerTokenizer, LongformerForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import os

In [ ]:
from transformers import LongformerModel
import torch.nn as nn

class LongformerForRegression(nn.Module):
  def __init__(self, model_name, num_outputs=2):
      super(LongformerForRegression, self).__init__()
      self.longformer = LongformerModel.from_pretrained(model_name)
      self.regressor = nn.Linear(self.longformer.config.hidden_size, num_outputs)

  def forward(self, input_ids, attention_mask):
      outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
      # Mean pooling (luăm media pe dimensiunea de secvență)
      sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
      mask = attention_mask.unsqueeze(-1).expand(sequence_output.size()).float()
      pooled_output = torch.sum(sequence_output * mask, dim=1) / torch.sum(mask, dim=1)
      predictions = self.regressor(pooled_output)
      return predictions

  def save_pretrained(self, save_directory):
      if not os.path.exists(save_directory):
          os.makedirs(save_directory)
      # self.bert.save_pretrained(save_directory)
      self.longformer.save_pretrained(save_directory)
      torch.save(self.regressor.state_dict(), os.path.join(save_directory, "regressor.pt"))

  @classmethod
  def from_pretrained(cls, load_directory, model_name, num_labels=2):

      model = cls(model_name, num_labels)
      # model.bert = AutoModel.from_pretrained(load_directory)
      model.longformer = LongformerModel.from_pretrained(load_directory)
      regressor_path = os.path.join(load_directory, "regressor.pt")
      model.regressor.load_state_dict(torch.load(regressor_path))

      return model

class BertRegressionModel(torch.nn.Module):

    def __init__(self, model_name, num_labels=2):
        super(BertRegressionModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.regressor = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        predictions = self.regressor(pooled_output)
        return predictions

class SummaryDataset(Dataset):
    def __init__(self, summaries, prompts,text ,scores_content, scores_wording, tokenizer, max_length=256):
        self.summaries = summaries
        self.prompts = prompts
        self.scores_content = scores_content
        self.scores_wording = scores_wording
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.text = text

    def __len__(self):
        return len(self.summaries)

    def __getitem__(self, idx):
        summary = self.summaries[idx]
        prompt = self.prompts[idx]
        text = self.text[idx]
        score_content = self.scores_content[idx]
        score_wording = self.scores_wording[idx]
        combined_text = f"Prompt: {prompt} [SEP] Text:{text} [SEP] Summary: {summary}"
        inputs = self.tokenizer(
            combined_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "scores": torch.tensor([score_content, score_wording], dtype=torch.float),
        }

def main():
    MODEL_NAME = "allenai/longformer-base-4096"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    model = LongformerForRegression(MODEL_NAME, num_outputs=2)
    # model = BertRegressionModel(MODEL_NAME, num_labels=2)
    # 1. Citim datele din CSV
    summaries_df = pd.read_csv(r"/content/gdrive/MyDrive/Summary_scoring/Baze_date/CLASSE_Corpus/summaries.csv")
    prompts_df = pd.read_csv(r"/content/gdrive/MyDrive/Summary_scoring/Baze_date/CLASSE_Corpus/prompts.csv")

    # Aliniem datele pe baza prompt_id
    merged_df = pd.merge(summaries_df, prompts_df, on="prompt_id")
    summaries = merged_df["text"].tolist()
    text = merged_df["prompt_text"].tolist()
    prompts = merged_df["prompt_question"].tolist()
    scores_content = merged_df["content"].tolist()
    scores_wording = merged_df["wording"].tolist()
    # 2. Preprocesarea datelor
    # Crearea datasetului si dataloader-ului
    dataset = SummaryDataset(summaries, prompts, text, scores_content, scores_wording, tokenizer)
    subset_size = int(0.5 * len(dataset))
    subset_dataset, remaining_dataset = random_split(dataset, [subset_size, len(dataset) - subset_size])

    test_size = int(0.2 * len(subset_dataset))
    train_size = len(subset_dataset) - test_size
    train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True, num_workers=2)
    test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False, num_workers=2)

    # 3. Configuram optimizatorul si criteriul de pierdere
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    criterion = torch.nn.MSELoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # 4. Loop de antrenare
    EPOCHS = 1
    model.train()
    for epoch in range(EPOCHS):
        epoch_start_time = time.time()
        epoch_loss = 0
        for batch_idx, batch in enumerate(train_dataloader):
            batch_start_time = time.time()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            scores = batch["scores"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = outputs  # dim [batch_size, 2]

            loss = criterion(predictions, scores)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            print(
                f"Batch {batch_idx + 1}/{len(train_dataloader)} processed in {time.time() - batch_start_time:.2f} seconds.")
            if (batch_idx + 1) % 10 == 0:
              print(f"Target: {scores.cpu().numpy()}")
              print(f"Prezis: {predictions.cpu().detach().numpy()}")

        avg_loss = epoch_loss / len(train_dataloader)
        print(
            f"Epoch {epoch + 1}/{EPOCHS} completed in {time.time() - epoch_start_time:.2f} seconds. Average Loss: {avg_loss:.4f}")
    print("Model fine-tuned salvat cu succes!")

    model.save_pretrained(r"/content/gdrive/MyDrive/Summary_scoring/fine_tuned_model")
    tokenizer.save_pretrained(r"/content/gdrive/MyDrive/Summary_scoring/fine_tuned_model")
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for i, batch in enumerate(test_dataloader):
          input_ids = batch["input_ids"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          scores = batch["scores"].to(device)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          predictions = outputs.cpu().numpy()
          targets = scores.cpu().numpy()
          print(f"Iteration {i+1}")

          all_predictions.extend(predictions)
          all_targets.extend(targets)

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    mse_content = mean_squared_error(all_targets[:, 0], all_predictions[:, 0])
    mse_wording = mean_squared_error(all_targets[:, 1], all_predictions[:, 1])

    print(f"MSE for Content: {mse_content:.4f}")
    print(f"MSE for Wording: {mse_wording:.4f}")

if __name__ == "__main__":
    main()


Batch 1/1495 processed in 0.68 seconds.
Batch 2/1495 processed in 0.60 seconds.
Batch 3/1495 processed in 0.60 seconds.
Batch 4/1495 processed in 0.67 seconds.
Batch 5/1495 processed in 0.60 seconds.
Batch 6/1495 processed in 0.60 seconds.
Batch 7/1495 processed in 0.60 seconds.
Batch 8/1495 processed in 0.60 seconds.
Batch 9/1495 processed in 0.60 seconds.
Batch 10/1495 processed in 0.60 seconds.
Target: [[ 1.6907398  1.461055 ]
 [-1.5471632 -1.4612448]
 [ 0.3606764  0.5009105]]
Prezis: [[-0.03683064 -0.00460844]
 [-0.04463234 -0.02219144]
 [-0.01527555  0.0092564 ]]
Batch 11/1495 processed in 0.60 seconds.
Batch 12/1495 processed in 0.61 seconds.
Batch 13/1495 processed in 0.60 seconds.
Batch 14/1495 processed in 0.60 seconds.
Batch 15/1495 processed in 0.60 seconds.
Batch 16/1495 processed in 0.61 seconds.
Batch 17/1495 processed in 0.60 seconds.
Batch 18/1495 processed in 0.60 seconds.
Batch 19/1495 processed in 0.60 seconds.
Batch 20/1495 processed in 0.60 seconds.
Target: [[-2.46

KeyboardInterrupt: 

In [ ]:
from transformers import LongformerModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import time

# Definirea modelului pentru regresie
# class LongformerForRegression(nn.Module):
#     def __init__(self, model_name, num_outputs=2):
#         super(LongformerForRegression, self).__init__()
#         self.longformer = LongformerModel.from_pretrained(model_name)
#         self.regressor = nn.Linear(self.longformer.config.hidden_size, num_outputs)
#         # Inițializare pentru regressor
#         nn.init.xavier_uniform_(self.regressor.weight)
#         nn.init.zeros_(self.regressor.bias)

#     def forward(self, input_ids, attention_mask):
#         outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
#         sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
#         mask = attention_mask.unsqueeze(-1).expand(sequence_output.size()).float()
#         pooled_output = torch.sum(sequence_output * mask, dim=1) / torch.sum(mask, dim=1)
#         predictions = self.regressor(pooled_output)
#         return predictions

#     def save_pretrained(self, save_directory):
#         if not os.path.exists(save_directory):
#             os.makedirs(save_directory)
#         self.longformer.save_pretrained(save_directory)
#         torch.save(self.regressor.state_dict(), os.path.join(save_directory, "regressor.pt"))


#     def from_pretrained(cls, load_directory, model_name, num_outputs=2):
#         model = cls(model_name, num_outputs)
#         model.longformer = LongformerModel.from_pretrained(load_directory)
#         regressor_path = os.path.join(load_directory, "regressor.pt")
#         model.regressor.load_state_dict(torch.load(regressor_path))
#         return model
class LongformerForRegression(nn.Module):
    class LongformerForRegression(nn.Module):
    def __init__(self, model_name, num_outputs=2, hidden_dim=256):
        super(LongformerForRegression, self).__init__()
        self.longformer = LongformerModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(p=0.1)
        # Head mai complex
        self.regressor = nn.Sequential(
            nn.Linear(self.longformer.config.hidden_size, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(hidden_dim, num_outputs)
        )
        # Inițializare pentru regressor
        for layer in self.regressor:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, input_ids, attention_mask):
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
        mask = attention_mask.unsqueeze(-1).expand(sequence_output.size()).float()
        pooled_output = torch.sum(sequence_output * mask, dim=1) / torch.sum(mask, dim=1)
        pooled_output = self.dropout(pooled_output)
        predictions = self.regressor(pooled_output)
        return predictions

    def save_pretrained(self, save_directory):
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        self.longformer.save_pretrained(save_directory)
        torch.save(self.regressor.state_dict(), os.path.join(save_directory, "regressor.pt"))

    @classmethod
    def from_pretrained(cls, load_directory, model_name, num_outputs=2):
        model = cls(model_name, num_outputs)
        model.longformer = LongformerModel.from_pretrained(load_directory)
        regressor_path = os.path.join(load_directory, "regressor.pt")
        model.regressor.load_state_dict(torch.load(regressor_path))
        return model
# Definirea datasetului personalizat
class SummaryDataset(Dataset):
    def __init__(self, summaries, prompts, texts, scores_content, scores_wording, tokenizer, max_length=256):
        self.summaries = summaries
        self.prompts = prompts
        self.texts = texts
        self.scores_content = scores_content
        self.scores_wording = scores_wording
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.summaries)

    def __getitem__(self, idx):
        summary = self.summaries[idx]
        prompt = self.prompts[idx]
        text = self.texts[idx]
        score_content = self.scores_content[idx]
        score_wording = self.scores_wording[idx]

        combined_text = f"Prompt: {prompt} [SEP] Text: {text} [SEP] Summary: {summary}"
        inputs = self.tokenizer(
            combined_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "scores": torch.tensor([score_content, score_wording], dtype=torch.float),
        }

# Funcția principală
def main():
    MODEL_NAME = "allenai/longformer-base-4096"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = LongformerForRegression(MODEL_NAME, num_outputs=2)

    # Citirea și pregătirea datelor
    summaries_df = pd.read_csv(r"/content/gdrive/MyDrive/Summary_scoring/Baze_date/CLASSE_Corpus/summaries.csv")
    prompts_df = pd.read_csv(r"/content/gdrive/MyDrive/Summary_scoring/Baze_date/CLASSE_Corpus/prompts.csv")

    # Alinierea datelor pe baza prompt_id
    merged_df = pd.merge(summaries_df, prompts_df, on="prompt_id")
    summaries = merged_df["text"].tolist()
    texts = merged_df["prompt_text"].tolist()
    prompts = merged_df["prompt_question"].tolist()
    scores_content = merged_df["content"].tolist()
    scores_wording = merged_df["wording"].tolist()

    # Crearea datasetului și a dataloader-ului
    dataset = SummaryDataset(summaries, prompts, texts, scores_content, scores_wording, tokenizer, max_length=512)
    subset_size = int(0.5 * len(dataset))
    subset_dataset, remaining_dataset = random_split(dataset, [subset_size, len(dataset) - subset_size])

    test_size = int(0.2 * len(subset_dataset))
    train_size = len(subset_dataset) - test_size
    train_dataset, test_dataset = random_split(subset_dataset, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True, num_workers=2)
    test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False, num_workers=2)

    # Configurarea optimizatorului și criteriului de pierdere
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
    criterion = torch.nn.HuberLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Loop de antrenare
    EPOCHS = 3
    model.train()
    for epoch in range(EPOCHS):
      epoch_start_time = time.time()
      epoch_loss = 0
      for batch_idx, batch in enumerate(train_dataloader):
          batch_start_time = time.time()
          input_ids = batch["input_ids"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          scores = batch["scores"].to(device)

          optimizer.zero_grad()
          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          predictions = outputs  # dim [batch_size, 2]

          loss = criterion(predictions, scores)
          loss.backward()
          optimizer.step()

          epoch_loss += loss.item()

          # Afișare pierdere la fiecare pas
          print(
              f"Epoch {epoch + 1}/{EPOCHS}, Batch {batch_idx + 1}/{len(train_dataloader)}, "
              f"Loss: {loss.item():.4f}, Batch time: {time.time() - batch_start_time:.2f} seconds."
          )

          if (batch_idx + 1) % 10 == 0:  # Afișare la fiecare 10 batch-uri
              print(f"Target: {scores.cpu().numpy()}")
              print(f"Prezis: {predictions.cpu().detach().numpy()}")

      avg_loss = epoch_loss / len(train_dataloader)
      print(
          f"Epoch {epoch + 1}/{EPOCHS} completed in {time.time() - epoch_start_time:.2f} seconds. "
          f"Average Loss: {avg_loss:.4f}"
      )

    model.save_pretrained(r"/content/gdrive/MyDrive/Summary_scoring/fine_tuned_model")
    tokenizer.save_pretrained(r"/content/gdrive/MyDrive/Summary_scoring/fine_tuned_model")

    # Evaluare pe setul de testare
    model.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for i, batch in enumerate(test_dataloader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            scores = batch["scores"].to(device)

            predictions = model(input_ids=input_ids, attention_mask=attention_mask)
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(scores.cpu().numpy())

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    mse_content = mean_squared_error(all_targets[:, 0], all_predictions[:, 0])
    mse_wording = mean_squared_error(all_targets[:, 1], all_predictions[:, 1])

    print(f"MSE for Content: {mse_content:.4f}")
    print(f"MSE for Wording: {mse_wording:.4f}")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Datele de ieșire de afișat au fost trunchiate la ultimele 5000 linii.
 [ 2.1126723   2.3848686 ]]
Prezis: [[-0.30875826 -0.4724705 ]
 [-0.29781988 -0.45393953]
 [-0.30387717 -0.46378168]]
Epoch 1/3, Batch 1361/1495, Loss: 0.9316, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1362/1495, Loss: 0.2020, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1363/1495, Loss: 0.5814, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1364/1495, Loss: 0.4145, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1365/1495, Loss: 0.1500, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1366/1495, Loss: 0.2522, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1367/1495, Loss: 0.3374, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1368/1495, Loss: 0.4520, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1369/1495, Loss: 0.6666, Batch time: 0.61 seconds.
Epoch 1/3, Batch 1370/1495, Loss: 0.4628, Batch time: 0.61 seconds.
Target: [[ 0.5679749   0.96906245]
 [-0.766094   -0.3254376 ]
 [-1.5471632  -1.4612448 ]]
Prezis: [[-0.17197189 -0.16282113]
 [-0.16